# **데이터 가져오기**

In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

In [2]:
seed = 1

batch_size = 64
test_batch_size = 64

no_cuda = False

In [3]:
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train = True, download = True,
                  transform = transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081))
                  ])),
    batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform= transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=test_batch_size, shuffle=True)

# **Model**

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2 ,2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

# **Optimization**

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.5)
params = list(model.parameters())
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# **Training**

In [8]:
epochs = 1
log_interval = 100
for epoch in range(1, epochs+1):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} {}/{} ({:.0f}%)\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))

Train Epoch: 1 0/60000 (0%)	Loss: 0.473569
Train Epoch: 1 6400/60000 (11%)	Loss: 0.712280
Train Epoch: 1 12800/60000 (21%)	Loss: 0.536166
Train Epoch: 1 19200/60000 (32%)	Loss: 0.501692
Train Epoch: 1 25600/60000 (43%)	Loss: 0.243773
Train Epoch: 1 32000/60000 (53%)	Loss: 0.407195
Train Epoch: 1 38400/60000 (64%)	Loss: 0.298076
Train Epoch: 1 44800/60000 (75%)	Loss: 0.262569
Train Epoch: 1 51200/60000 (85%)	Loss: 0.490643
Train Epoch: 1 57600/60000 (96%)	Loss: 0.282653


# Evaluation
- 앞에서 model.train() 모드로 변한 것처럼 평가 할 때는 model.eval()로 설정
 - Batch Normaliztion이나 Drop Out같은 Layer들을 잠금

In [10]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engine, 즉 backpropagation 이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임

In [12]:
test_loss = 0
correct = 0

with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    test_loss += F.nll_loss(output, target, reduction='sum').item()
    
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(target.view_as(pred)).sum().item()

In [17]:
output.shape

torch.Size([64, 10])

In [16]:
pred.shape

torch.Size([64, 1])

In [18]:
target.view_as(pred).shape

torch.Size([64, 1])

In [13]:
test_loss

16.042293548583984

In [14]:
correct

59

In [20]:
pred.eq(target.view_as(pred)).sum().item() / 64

0.921875

In [21]:
test_loss /= len(test_loader.dataset)

In [22]:
test_loss

0.0016042293548583985

# **전체 테스트 이미지학습 결과**

In [23]:
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
test_loss /= len(test_loader.dataset)

print('\n Test set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))


 Test set: Average Loss: 0.3005, Accuracy: 9097/10000 (91%)



# **전체 데이터 학습과 테스트 과정**

In [24]:
epochs = 1
log_interval = 100
for epoch in range(1, epochs+1):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} {}/{} ({:.0f}%)\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))
    
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    test_loss /= len(test_loader.dataset)

    print('\n Test set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100 * correct / len(test_loader.dataset)))

Train Epoch: 1 0/60000 (0%)	Loss: 0.500561
Train Epoch: 1 6400/60000 (11%)	Loss: 0.419360
Train Epoch: 1 12800/60000 (21%)	Loss: 0.307164
Train Epoch: 1 19200/60000 (32%)	Loss: 0.279458
Train Epoch: 1 25600/60000 (43%)	Loss: 0.220473
Train Epoch: 1 32000/60000 (53%)	Loss: 0.171871
Train Epoch: 1 38400/60000 (64%)	Loss: 0.329787
Train Epoch: 1 44800/60000 (75%)	Loss: 0.409632
Train Epoch: 1 51200/60000 (85%)	Loss: 0.164350
Train Epoch: 1 57600/60000 (96%)	Loss: 0.342703

 Test set: Average Loss: 0.2270, Accuracy: 9324/10000 (93%)

